### export_usa_choropleth_fips_maps

In [8]:
# Init
# %pip install plotly
# %pip install nbformat
# --upgrade nbformat

from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import seaborn as sns

export_df = pd.read_csv('../unzipped_shp_and_shx_shape_files/export.csv', na_values=".")


In [9]:
export_df.head()

,fips,county_zip_file_name,roads_length_m,intersections,countyname,population,state,county,area_land_sqm,area_water_sqm,...,B11016_family_households_7_or_more,B11016_family_households_total,B13002_fertile_women_given_birth,B13002_fertile_women_total,area_land_sqkm,roads_length_km,intersects_per_road_length,road_length_per_intersect,population_density,intersects_per_population_density
0,1001,tl_2020_01001_roads.zip,2658136.223,4926,"Autauga County, Alabama",58805,1,1,1.539631e+09,2.567754e+07,...,129,15103,743,13737,1539.631461,2658.136223,1.853178,0.539614,38.194205,70.780667
1,1003,tl_2020_01003_roads.zip,6818407.996,17393,"Baldwin County, Alabama",231767,1,3,4.117725e+09,1.132887e+09,...,583,56092,1892,46768,4117.724893,6818.407996,2.550889,0.392020,56.285207,143.577301
2,1005,tl_2020_01005_roads.zip,2025010.112,2918,"Barbour County, Alabama",25223,1,5,2.292160e+09,5.052321e+07,...,57,6083,339,4756,2292.160151,2025.010112,1.440980,0.693972,11.004030,15.856593
3,1007,tl_2020_01007_roads.zip,1957160.039,3039,"Bibb County, Alabama",22293,1,7,1.612189e+09,9.572302e+06,...,145,5276,530,4709,1612.188713,1957.160039,1.552760,0.644014,13.827786,21.471234
4,1009,tl_2020_01009_roads.zip,2615906.457,4195,"Blount County, Alabama",59134,1,9,1.670259e+09,1.486028e+07,...,279,15035,511,12551,1670.259093,2615.906457,1.603651,0.623577,35.404088,56.775788


In [15]:

export_df.loc[export_df.index[1], 'fips_str']

export_df.iloc[0,1]


'tl_2020_01001_roads.zip'

In [16]:
# row first

for idx in export_df.index:
    export_df.loc[export_df.index[idx], 'fips_str'] = str(export_df.loc[export_df.index[idx], 'fips'])
    if len(export_df.loc[export_df.index[idx], 'fips_str']) == 4:
        export_df.loc[export_df.index[idx], 'fips_str'] = '0' + export_df.loc[export_df.index[idx], 'fips_str']

export_df['fips_str'].head()


AttributeError: 'numpy.int64' object has no attribute 'to_string'

### Maps based on age group density per household

##### plotly documentation link & example code

In [ ]:
#### https://plotly.com/python/choropleth-maps/
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
export_df['fips_str'].head()

##### attempt at export_df graph

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.express as px

fig = px.choropleth(export_df, geojson=counties, locations='fips_str', color='B09001_population_3_to_4',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:

export_df['population_density/intersect_density'] = export_df['population_density'] / export_df['intersects_per_road_length']
export_df['log_population_density'] = np.log(export_df['population_density'])
export_df['log_intersect_density'] = np.log(export_df['intersects_per_road_length'])

export_df['log_population_density/log_intersect_density'] = export_df['log_population_density'] / export_df['log_intersect_density']

export_df['target'] = export_df['log_population_density/log_intersect_density']


In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.express as px

fig = px.choropleth(export_df, geojson=counties, locations='fips_str', color='log_population_density',
                           color_continuous_scale="Viridis",
                           range_color=(-2, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.express as px

fig = px.choropleth(export_df, geojson=counties, locations='fips_str', color='log_intersect_density',
                           color_continuous_scale="Viridis",
                           range_color=(-1, 2.5),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# DOES INTERSECT DENSITY PROVIDE A BETTER METRIC FOR WHICH ECONOMIC GROWHT OCCURS